In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from math import ceil
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
import string
from math import log10
from scipy.stats import norm
import re
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
columns = ['tweet_id','timestamp','tweet_text','user_id',
           'tweet_coords','tweet_coords_list','tweet_long','tweet_lat','location',
           'enc_url','tweet_lang','hashtags']
tweet_full = pd.read_csv(r'./tweetCoords.csv',
                         header=None,
                         names=columns,
                         parse_dates=[1],
                         infer_datetime_format=True,
                         index_col='timestamp')

In [3]:
tweet_full_en = tweet_full[tweet_full['tweet_lang'] == 'en']

In [4]:
tweet_stops = stopwords.words('english')
tweet_tokenizer = TweetTokenizer(strip_handles=True,preserve_case=False,reduce_len=True)

def clean_tweet(tweet):
#     takes input string and converts or removes characters depending on settings.
#     returns a string
#     convert case:
    tweet = tweet.lower()
#     remove URLs:
    tweet = re.sub('https?://\S+','',tweet)
#     remove @mentions, including those with a leading '-' or '.' : 
    tweet = re.sub('[-\.]?@\w+','',tweet)
#     remove punctuation, but not hashtags:
    tweet = tweet.translate(tweet.maketrans('','',string.punctuation.replace("#","")))
#     remove non-hashtag '#'.
    tweet = re.sub('#\B','',tweet)
#     remove 'amp', 'gt', 'lt', indicating decoded ampersand, greater-than, less-than characters
    tweet = re.sub(r'\b(amp|gt|lt)\b','',tweet)
#     drop numbers and words of < 4 characters.
    tweet = re.sub(r'\b\w{1,3}\b','',tweet)
    tweet = re.sub(r'\b\d+\b','',tweet)
    return tweet

def tokens_no_stopwords(tweet_as_string):
#     wrapper function that combines the tokenizer, cleaner, and stopword removal.
#     takes a string and returns a list of strings
    cleaned_tweet = clean_tweet(tweet_as_string)
    tweet_as_tokens = tweet_tokenizer.tokenize(cleaned_tweet)
    tweet_no_stops = [word for word in tweet_as_tokens if word not in tweet_stops]
    
    return tweet_no_stops

In [5]:
search_term = "irma"

In [11]:
tweet_date = pd.to_datetime("2017-09-10 00:00:00")
date_delta = pd.Timedelta("72HR")

In [12]:
tweet_text = tweet_full_en.loc[tweet_date:tweet_date + date_delta,"tweet_text"]

tweets_tokens = tweet_text.apply(tokens_no_stopwords)

vector_model = Word2Vec(tweets_tokens, min_count=1, window=5, workers=1, size=100, seed=1, sg=1)

word_matrix = vector_model.wv[vector_model.wv.vocab]
vector_model.train(tweets_tokens, total_examples=len(tweet_text), epochs=10)

(2459097, 2638770)

In [9]:
def ScoreTweetFromVectors(tweet,vector_set):
    tweet_as_terms = tokens_no_stopwords(tweet)
#     initialize vector as 100 dimension vector of zeroes.
    score_matrix = np.zeros(100,) 
#     iterate over each word after processing. If the word is in the vocabulary,
#     add its vector's value to the score matrix.
#     this essentially treats a word not in the vocabulary as a zero-vector.
    for i in tweet_as_terms:
        if i in vector_set.wv.vocab:
            score_matrix = np.add(score_matrix,vector_set.wv.get_vector(i))
#     if the number of words remaining in the tweet after processing is equal to zero, return zero.
#     otherwise, take the dot product of the score vector, and the vector of the search term.
    if len(tweet_as_terms) > 0:
        score = np.dot(score_matrix,vector_set.wv.get_vector(search_term))
    else:
        score = 0
    return score

In [21]:
tweet_block = tweet_full_en.loc[tweet_date:tweet_date + date_delta,['tweet_id','tweet_text']].copy()

In [22]:
tweet_block.head()

,tweet_id,tweet_text
timestamp,,
2017-09-10 00:00:01,906668545542680576,"Wind 2.0 mph N. Barometer 29.880 in, Steady. T..."
2017-09-10 00:00:03,906668555185291265,There is always beauty regardless the circumst...
2017-09-10 00:00:03,906668556493889536,#Carpool #Orlando - gt #Orlando #KIrkmanRoad ...
2017-09-10 00:00:06,906668570079309830,@avictoria_x nah i m chillin my eyes never get...
2017-09-10 00:00:08,906668576056246278,@jvnvy @sza That s how I feel bout seein travis


In [23]:
tweet_block['raw_score']= tweet_block['tweet_text'].apply(ScoreTweetFromVectors,args=(vector_model,))

In [26]:
tweet_block['mm_score'] = (tweet_block['raw_score'] - tweet_block['raw_score'].min())* 100 / (tweet_block['raw_score'].max() - tweet_block['raw_score'].min())

In [28]:
tweet_block.to_csv('three_day_scores.csv')